https://docs.databricks.com/_static/notebooks/structured-streaming-kafka.html

This is a WordCount example with the following

Kafka as a Structured Streaming Source
Stateful operation (groupBy) to calculate running counts
Requirements
An Apache Kafka 0.10.0 (or above) deployment

In [ ]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{explode, split}

In [2]:
val spark = SparkSession.builder().appName("SparkByExample").getOrCreate();

In [3]:
println("First SparkContext:")
println("APP Name :"+spark.sparkContext.appName);
println("Deploy Mode :"+spark.sparkContext.deployMode);
println("Master :"+spark.sparkContext.master);

First SparkContext:
APP Name :Apache Toree
Deploy Mode :client
Master :spark://master:7077


In [5]:
// Subscribe to 1 topic
val df = spark.readStream.
format("kafka").
option("kafka.bootstrap.servers", "localhost:9092,10.96.0.20:9096"). // comma separated list of broker:host
option("subscribe", "sample_topic,android"). // comma separated list of topics
option("startingOffsets", "latest"). // read data from the end of the stream
load()

Name: java.lang.ClassNotFoundException
Message: Failed to find data source: kafka. Please find packages at https://cwiki.apache.org/confluence/display/SPARK/Third+Party+Projects
StackTrace:   at org.apache.spark.sql.execution.datasources.DataSource.lookupDataSource(DataSource.scala:148)
  at org.apache.spark.sql.execution.datasources.DataSource.providingClass$lzycompute(DataSource.scala:79)
  at org.apache.spark.sql.execution.datasources.DataSource.providingClass(DataSource.scala:79)
  at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:218)
  at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:80)
  at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo(DataSource.scala:80)
  at org.apache.spark.sql.execution.streaming.StreamingRelation$.apply(StreamingRelation.scala:30)
  at org.apache.spark.sql.streaming.DataStreamReader.load(DataStreamReader.scala:124)
  ... 42 elided
Caused by: java.lang.Cl

In [ ]:
// split lines by whitespace and explode the array as rows of `word`
val df = kafka.select(explode(split($"value".cast("string"), "\\s+")).as("word")).
groupBy($"word").count

In [7]:
// follow the word counts as it updates
display(df.select($"word", $"count"))

Name: Unknown Error
Message: lastException: Throwable = null
<console>:19: error: not found: value df
       df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").as[(String, String)]
       ^
StackTrace: 

Try it yourself
The Kafka Source also includes the ingestion timestamp of records. Try counting the words by the ingestion time window as well.

Note: Kafka provides timestamps in milliseconds. In order to be able to cast the long value to timestamp, you will have to divide the timestamp by 1000.